# border detection success

In [2]:
import cv2
import numpy as np
import pyautogui
import time

def get_screen():
    x, y, width, height = 60, 65, 480, 432  # Emulator window region
    screenshot = pyautogui.screenshot(region=(x, y, width, height))
    return cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

def detect_game_border(screen):
    screen_height, screen_width, _ = screen.shape
    border_color_bgr = np.array([68, 68, 72], dtype=np.uint8) # #484444 的 BGR 值

    top_border = 0
    while top_border < screen_height and np.all(screen[top_border, :] == border_color_bgr):
        top_border += 1

    bottom_border = screen_height - 1
    while bottom_border >= 0 and np.all(screen[bottom_border, :] == border_color_bgr):
        bottom_border -= 1

    left_border = 0
    while left_border < screen_width and np.all(screen[:, left_border] == border_color_bgr):
        left_border += 1

    right_border = screen_width - 1
    while right_border >= 0 and np.all(screen[:, right_border] == border_color_bgr):
        right_border -= 1

    # 檢查是否找到了有效的非邊界區域
    if top_border < bottom_border and left_border < right_border:
        return left_border, top_border, right_border - left_border + 1, bottom_border - top_border + 1
    else:
        return None

if __name__ == "__main__":
    print("Starting game border detection. Press 'q' to quit.")
    time.sleep(2)

    while True:
        screen = get_screen()
        game_border = detect_game_border(screen)

        overlay = screen.copy()
        if game_border:
            x, y, w, h = game_border
            cv2.rectangle(overlay, (x, y), (x + w - 1, y + h - 1), (0, 255, 0), 2) # Green rectangle for border
            print(f"Detected game border: x={x}, y={y}, width={w}, height={h}")
        else:
            print("Game border not detected.")

        cv2.imshow("Game Border Detection", overlay)

        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()